In [1]:
import os
import pandas as pd
import requests
from tqdm import tqdm
import re
import urllib.parse

In [ ]:
from '../src/utils/data)cleaning.py' import replace_unicode_placeholders

ModuleNotFoundError: No module named 'replace_unicode_placeholders'

In [3]:
df = pd.read_csv("../datasets/raw/all_data.csv")
df = all_data_df.loc[:, ~all_data_df.columns.str.contains('^Unnamed')]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20015 entries, 0 to 20014
Data columns (total 54 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0.1        20015 non-null  int64  
 1   Unnamed: 0          3171 non-null   float64
 2   author              20015 non-null  object 
 3   comments            16844 non-null  float64
 4   country             20015 non-null  object 
 5   crawled             16844 non-null  object 
 6   domain_rank         20015 non-null  int64  
 7   id                  4903 non-null   float64
 8   language            16844 non-null  object 
 9   likes               16844 non-null  float64
 10  main_img_url        20015 non-null  object 
 11  ord_in_thread       16844 non-null  float64
 12  participants_count  16844 non-null  float64
 13  published           13514 non-null  object 
 14  replies_count       16844 non-null  float64
 15  shares              16844 non-null  float64
 16  site

In [37]:
# Create a directory to save images

def sanitize_filename(filename, max_length=255):
    # Decode URL encoding
    filename = urllib.parse.unquote(filename)
    # Replace non-alphanumeric characters with underscores
    sanitized_filename = re.sub(r'[^a-zA-Z0-9\-_\.]', '_', filename)
    # Truncate the filename to fit within the max length allowed by the filesystem
    if len(sanitized_filename) > max_length:
        sanitized_filename = sanitized_filename[:max_length]
    return sanitized_filename
    
os.makedirs('images', exist_ok=True)

# Initialize a list to keep track of failed downloads
failed_downloads = []

# Define headers to mimic a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# Function to create image filename from URL (extracting part after '.com/' and before '.jpg')
def get_image_filename(url):
    # Use regular expression to find the part after '.com/' and before '.jpg'
    match = re.search(r'\.com/(.*?)(?=\.[a-zA-Z]{3,4})', url)  # Match part between '.com/' and file extension
    if match:
        return match.group(1).replace('/', '_')  # Replace '/' with '_' for safe filenames
    return None  # Return None if no match is found


start_index = 7665+6867+1944

df_subset = df.iloc[start_index:]
# Iterate through the DataFrame with a progress bar
for index, row in tqdm(df_subset.iterrows(), total=len(df_subset)):
    if index < start_index:
        continue  # Skip already processed rows
    image_name = get_image_filename(row['main_img_url'])  # Extract image name from URL
   
    if image_name is None:
        print(f"Skipping invalid URL: {row['main_img_url']}")
        continue  # Skip if no valid filename can be derived from the URL

    image_path = f'images/{image_name}.jpg'  # Path to save the image

    # Check if the image already exists
    if os.path.exists(image_path):
        print(f"Image {image_name}.jpg already exists. Skipping...")
        continue

    url = row['main_img_url']
    
    try:
        # Request the URL with the custom headers
        response = requests.get(url, headers=headers, stream=True, timeout=10)

        # Check if the status code is 403 (Forbidden)
        if response.status_code == 403:
            print(f"Forbidden: Skipping {image_name}.jpg (403 status)")
            continue  # Skip the current image if 403

        
        # Check if the request was successful (status code 200)
        response.raise_for_status()  # Raise an error for any non-200 status code
        try:
            with open(image_path, 'wb') as out_file:
                out_file.write(response.content)
            print(f"Downloaded {image_name}.jpg")
        
        except FileNotFoundError:
            print(f"FileNotFoundError: Skipping {image_name}.jpg due to invalid path or filename length.")
            continue  # Skip the problematic image if file cannot be saved due to path issues

    except requests.RequestException as e:
        print(f"Failed to download {url}: {e}")
        failed_downloads.append({'image_name': image_name, 'image_url': url, 'error': str(e)})

# Save failed downloads to a CSV for review
if failed_downloads:
    failed_df = pd.DataFrame(failed_downloads)
    failed_df.to_csv('failed_downloads_2.csv', index=False)
    print("Some downloads failed. Details are saved in 'failed_downloads.csv'.")

  0%|▏                                                                                | 6/3539 [00:00<02:49, 20.84it/s]

Failed to download https://1.bp.blogspot.com/-mxA9tLhTows/WBn3PsPNJWI/AAAAAAAAUv8/_ccU5BmGq9sqsF2X4GzmBJ4eN7Fh2Qm4wCLcB/w1200-h630-p-nu/He%2BLoses%2BHis%2BPulse%2BFor%2B45%2BMinutes%252C%2BWakes%2BUp%2BAnd%2BReveals%2BTHIS%2BAfterlife%2BChilling%2BVision.jpg: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Image -sizvWwZwSCc_WBn0vJEPyLI_AAAAAAAAUvs_QeCly9dFcwA3GZG42PdzfmshtyJBU_89gCLcB_w1200-h630-p-nu_15%2BSecret%2BiPhone%2BCodes%2BAnd%2BTricks.jpg already exists. Skipping...
Image -wWO98mM4Jyw_WBs50Gqq0rI_AAAAAAAAUw0_00TJR9rMQj0J36K3UiRN-7rONEOu25sXwCLcB_w1200-h630-p-nu_The%2BBiggest%2BRecord-Breaking%2BSupermoon%2BIn%2BNearly%2B70%2BYears.%2BDon%2527t%2BMiss%2BTHIS%2521.jpg already exists. Skipping...
Image -00tmZOlVUK8_WBs1SgjJwlI_AAAAAAAAUwg_V9DKRtG90y0jHdTObrfdhhW4a-vt0ed_wCLcB_w1200-h630-p-nu_She%2BSits%2BDown%2BNext%2BTo%2BA%2BGiant%2BWolf.%2BWatch%2BThe%2BMoment%2BWhen%2BTheir%2BEyes%2BMeet

  0%|▏                                                                               | 10/3539 [00:00<03:41, 15.91it/s]

FileNotFoundError: Skipping -8hL6t4faYJc_WCCfqHJT5FI_AAAAAAAAU1E_MbhfU6OHY_8HnZzeHW0IzkT5JJUiVv_YQCLcB_w1200-h630-p-nu_This%2BCilantro%2BRecipe%2BCan%2BRemove%2B80%2525%2BOf%2BHeavy%2BMetals%2BFrom%2BYour%2BBody%2BIn%2BJust%2B42%2BDays.jpg due to invalid path or filename length.
Downloaded -vtHjcq-4foY_WCCOz3x9JmI_AAAAAAAAU0M_6PHJGGF6pt0x9lUuJLIbLOx-Zh2DDBJ8QCLcB_w1200-h630-p-nu_Standing%2BRock%2BProtests%2BPolice%2BTurn%2BIn%2BBadges%2BRather%2BThan%2BIncite%2BViolence.jpg


  0%|▎                                                                               | 12/3539 [00:01<10:02,  5.86it/s]

...

Downloaded wp-content_uploads_2016_11_trump_security_screenshot.jpg
Skipping invalid URL: http://worldtruth.tv/wp-content/uploads/2016/10/Clintons-Maldives-1024x576.jpg
Skipping invalid URL: http://worldtruth.tv/wp-content/uploads/2016/10/WikiLeaks-List-Exposes-At-Least-65-Corporate-Presstitutes-Who-Colluded-to-Hide-Clinton-s-Crimes.png
Skipping invalid URL: http://worldtruth.tv/wp-content/uploads/2016/10/What-These-3-Parts-In-His-Little-Finger-Means-If-You-Have-It-<U+2026>-You-Will-Be-Surprised.jpg
Skipping invalid URL: http://worldtruth.tv/wp-content/uploads/2016/10/q4fj3e9jr2lc9yj1c3tn.jpeg
Skipping invalid URL: http://worldtruth.tv/wp-content/uploads/2016/10/36bf8087-1d6d-4d97-b32a-220698b1d963.jpg
Skipping invalid URL: http://worldtruth.tv/wp-content/uploads/2016/10/13zg758dogx6xab8kidn.jpeg
Skipping invalid URL: http://worldtruth.tv/wp-content/uploads/2016/10/image-20150923-2605-1068j24.jpg
Skipping invalid URL: http://worldtruth.tv/wp-content/uploads/2016/10/How-to-Remove-Turmer

  5%|███▋                                                                           | 167/3539 [00:41<04:29, 12.49it/s]

Downloaded wp-content_uploads_2014_09_axwell-1000x600.jpg
Downloaded wp-content_uploads_2016_11_talktohand-1000x600.jpg


100%|██████████████████████████████████████████████████████████████████████████████| 3539/3539 [00:44<00:00, 80.01it/s]

Failed to download http://cdns.yournewswire.com/wp-content/uploads/2016/10/Ash-Carter.jpg: HTTPConnectionPool(host='cdns.yournewswire.com', port=80): Max retries exceeded with url: /wp-content/uploads/2016/10/Ash-Carter.jpg (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001B5A9362840>: Failed to resolve 'cdns.yournewswire.com' ([Errno 11001] getaddrinfo failed)"))
Failed to download http://cdns.yournewswire.com/wp-content/uploads/2016/10/Hillary_nuclear_war_russia.jpg: HTTPConnectionPool(host='cdns.yournewswire.com', port=80): Max retries exceeded with url: /wp-content/uploads/2016/10/Hillary_nuclear_war_russia.jpg (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001B59F0DA390>: Failed to resolve 'cdns.yournewswire.com' ([Errno 11001] getaddrinfo failed)"))
Failed to download http://cdns.yournewswire.com/wp-content/uploads/2016/10/Russian-aircraft-carrier.jpg: HTTPConnectionPool(host='cdns.yournewswire.com', port=80): 

In [14]:
failed_df = pd.DataFrame(failed_downloads)
failed_df.to_csv('failed_downloads_7665.csv', index=False)